In [ ]:
from os import getenv
from connect_db import connect_db
con = connect_db(
    username=getenv("ORACLE_USERNAME"),
    password=getenv("ORACLE_PASSWORD"),
    host=getenv("ORACLE_HOST"),
    port=int(getenv("ORACLE_PORT"))
)
cur = con.cursor()

In [ ]:
import numpy as np

stmt = """
SELECT AIRCRAFT.AIRCRAFT_MODEL_ID, NUMBER_OF_SEAT
FROM AIRCRAFT
    JOIN AIRCRAFT_MODEL ON AIRCRAFT.AIRCRAFT_MODEL_ID = AIRCRAFT_MODEL.AIRCRAFT_MODEL_ID
"""
aircraft_model_ids = []
weights = []
for row in cur.execute(stmt):
    if row[1] < 10:
        weights.append(1)
    elif row[1] < 150:
        weights.append(2)
    elif row[1] < 350:
        weights.append(3)
    else:
        weights.append(2)
    aircraft_model_ids.append(row[0])
weights = np.array(weights) / sum(weights)

In [ ]:
stmt = "SELECT PILOT_ID FROM PILOT"
pilot_ids = [row[0] for row in cur.execute(stmt)]
pilot_ids

In [ ]:
import numpy as np
from typing import Iterable

def generate_flight_hours() -> int:
    hours = np.arange(2500, 25000, dtype=int)
    weights = np.concatenate((
        np.ones(2500, dtype=int) * 0.3 / 2500,
        np.ones(5000, dtype=int) * 0.4 / 5000,
        np.ones(5000, dtype=int) * 0.2 / 5000,
        np.ones(10000, dtype=int) * 0.1 / 10000,
    ))
    return np.random.choice(hours, p=weights, size=1).item()

def get_random_aircraft_model_ids() -> Iterable[str]:
    n = 0
    for i in range(len(aircraft_model_ids)):
        if np.random.randint(1, np.power(2, i) + 1) != 1:
            break
        n += 1
        
    yield from np.random.choice(aircraft_model_ids, p=weights, size=n)

In [ ]:
sql = "    INTO FLIGHT_HOUR (PILOT_ID, AIRCRAFT_MODEL_ID, FLIGHT_HOUR) VALUES('{}', '{}', {})"
for pilot_id in pilot_ids:
    for aircraft_model_id in get_random_aircraft_model_ids():
        print(sql.format(pilot_id,aircraft_model_id, generate_flight_hours()))

In [ ]:
con.close()